In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import mne, re, os
import torch
import pickle

from utils import *

In [3]:
SNR = 100.0
#PARCELLATION = 'aparc.a2009s'
PARCELLATION = 'HCPMMP1_combined'
DATA_PATH = '/home/williamtheodor/Documents/DL for EEG Classification/data/'

data_dict = load_mmidb_data_dict(DATA_PATH, PARCELLATION, SNR, chop=True)

In [23]:
data_dict['Alpha']['S092']['S092R02']

{'T0': array([[5.81118516e-21, 3.16614190e-20, 2.44434784e-20, 2.52641028e-20,
         5.00030927e-20, 7.98743224e-21, 1.56032258e-20, 4.74734847e-20,
         3.69776767e-20, 1.59769971e-20, 2.22434611e-20, 2.86912528e-20,
         1.05645796e-20, 6.06032673e-20, 3.52725192e-20, 2.16897836e-20,
         2.12075917e-20, 4.31496549e-20, 1.20943860e-20, 4.01034010e-20,
         9.16757421e-20, 2.26222363e-20, 1.51490834e-20, 5.68580079e-21,
         3.78652633e-20, 1.65970580e-20, 3.28278869e-20, 6.52458770e-20,
         5.73902742e-21, 1.44398729e-20, 6.89663169e-20, 4.41023281e-20,
         1.44198130e-20, 1.76407074e-20, 2.39888528e-20, 8.65022732e-21,
         7.97425526e-20, 3.37147281e-20, 2.03929765e-20, 1.27882366e-20,
         2.72036510e-20, 1.22432788e-20, 2.85796432e-20, 8.44020489e-20,
         2.58754067e-20, 1.10546398e-20],
        [8.58970729e-21, 3.81574096e-20, 3.46271622e-20, 4.43111105e-20,
         7.37477070e-20, 1.07640047e-20, 3.27946018e-20, 7.86568498e-20,
   

In [24]:
DATA_PATH_RAW = '../../data/eegmmidb (raw)/files/'
DATA_PATH_CONCEPTS = '../../data/baseline concepts '+PARCELLATION +'/'

bands = data_dict.keys()
patients = [key for key in data_dict['Alpha'].keys() if key not in ['S088', 'S089', 'S092', 'S104', 'S106']]
#runs = [key[-3:] for key in data_dict['Alpha'][patients[0]].keys()]
runs = ['R01', 'R02']

subjects_dir, subject, trans, src_path, bem_path = get_fsaverage()
labels = get_labels(subjects_dir, parcellation_name=PARCELLATION)
label_names = [label.name for label in np.array(labels).flatten()]

In [25]:
for label_idx in range(len(label_names)):
    # make directory if it doesn't exist
    if not os.path.exists(f'{DATA_PATH_CONCEPTS}{label_names[label_idx]}'):
        os.makedirs(f'{DATA_PATH_CONCEPTS}{label_names[label_idx]}')

In [26]:
NUMNER_PATIENTS = len(patients)
NUMBER_RUNS = len(runs)
NUMBER_WINDOWS = 15
NUMBER_CHANNELS = 20
NUMBER_SAMPLES = int(4*256)

band = 'Alpha'

X = np.zeros((NUMNER_PATIENTS, NUMBER_RUNS, NUMBER_WINDOWS, NUMBER_CHANNELS, NUMBER_SAMPLES))
Y = np.ones((NUMNER_PATIENTS, NUMBER_RUNS, NUMBER_WINDOWS)) * -1

for patient in tqdm(patients):
    for run in runs:

        FILE = DATA_PATH_RAW+f'{patient}/{patient}{run}.edf'

        raw = get_raw(FILE)

        raw_chops = get_raw_chops(raw)

        baseline_activity = data_dict[band][patient][patient+run]['T0'].mean(axis=0)

        for chop_idx, raw_chop in enumerate(raw_chops):
            raw_chop = pick_and_rename_channels(raw_chop)  

            activity = data_dict[band][patient][patient+run]['T0'][chop_idx]
            activity -= baseline_activity
            concept_idx = activity.argmax()
            #Y_temp_encoded = le.transform(Y_temp)  
            #print(DATA_PATH_CONCEPTS + label_names[concept_idx] + '/' + patient + run + '_' + label_names[concept_idx] + '.pkl')         

            try: 
                x = np.zeros((1, NUMBER_CHANNELS, NUMBER_SAMPLES))
                x[:,:19,:] = raw_chop.copy().get_data()[:,:NUMBER_SAMPLES].reshape(1,NUMBER_CHANNELS-1,NUMBER_SAMPLES)
                x[:,19,:] = np.ones((1, NUMBER_SAMPLES)) * -1  
                x = torch.from_numpy(x).float()
                
                picklePath = DATA_PATH_CONCEPTS + label_names[concept_idx] + '/' + patient + run + '_' + label_names[concept_idx] + '.pkl'
                with open(picklePath, 'wb') as handle:
                    pickle.dump(x, handle, protocol=pickle.HIGHEST_PROTOCOL)
            except:
                print('Error with file: ' + FILE,x.shape)

100%|██████████| 104/104 [00:55<00:00,  1.88it/s]
